<a href="https://colab.research.google.com/github/rquerino/Reverse-Game-of-Life/blob/master/reverse_game_of_life.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reverse game of life

![cellular automata](https://upload.wikimedia.org/wikipedia/commons/e/e5/Gospers_glider_gun.gif)

In [0]:
# R